<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-10 13:05:25
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 11.80 K (0.08%)
Current PnL: -29.06 L (-18.77%)
CY Booked + Current PnL: -14.77 L (-9.54%)
-------------------
Total profit:  1.20 L
Total loss:  -30.26 L
-------------------
Total Booked + Current PnL: 12.21 L (9.55%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.26%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.69 L (67.3%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.79%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,1.25,-17.00,20.60,0.09,17231.0,-17133.0,83644.0,94.24,40.0,M-SC,2.36,253.0,-0.99,0.62,9.89,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,2.01,-12.98,14.94,0.02,22694.0,-22659.0,151900.0,131.58,51.0,M-SC,16.60,234.0,-1.00,1.13,32.13,OX40N,NTT,PAINTS
43,ITC,452.00,0.56,-1.46,12.08,10.44,23823.0,-2924.0,197214.0,-41.15,47.0,X-LC,1.71,5.0,-0.12,1.47,4.02,X40,NTT,FMCG
50,MASFIN,398.61,0.39,-2.56,25.25,22.05,24107.0,-2505.0,95475.0,-15.99,55.0,H-SC,6.85,164.0,-0.10,0.71,38.78,XR,ATH,FINANCE
82,VOLTAS,1530.00,1.01,4.51,14.53,19.69,29116.0,8643.0,200385.0,-5.64,44.0,X-MC,2.69,78.0,0.30,1.49,11.61,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.0,4.72,-43.78,140.60,35.27,89759.0,-49709.0,63840.0,-57.75,33.0,L-SC,10.16,270.0,-0.55,0.48,56.25,XR,NTT,HOTELS
44,JCHAC,2282.0,3.19,-35.92,56.03,-0.01,45889.0,-45905.0,81900.0,16660.87,24.0,M-SC,4.10,233.0,-1.00,0.61,4.82,OX40N,NTT,AC
36,ICICIPRULI,790.0,3.11,6.60,22.86,30.97,43068.0,11661.0,188399.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.40,19.99,X40,ATH,INSURANCE
6,ASIANTILES,137.0,2.79,-17.31,119.69,81.67,92851.0,-16234.0,77576.0,6972.22,42.0,L-SC,14.32,271.0,-0.17,0.58,50.23,XR,NTT,CERAMICS
62,SATIN,274.0,2.46,-22.08,94.85,51.83,204202.0,-60995.0,215289.0,-30.35,35.0,H-SC,1.97,146.0,-0.30,1.60,4.88,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.10,-2.67,-37.33,122.76,39.61,78861.0,-38260.0,64240.0,-57.57,23.0,H-SC,26.26,135.0,-0.49,0.48,0.00,AR,ATH,MISC
61,ROUTE,2227.21,-2.54,-48.84,230.45,69.05,156877.0,-64992.0,68074.0,-58.67,46.0,H-SC,23.21,140.0,-0.41,0.51,4.98,SR,ATH,IT
23,DMART,5391.45,-2.38,-9.29,41.25,28.12,78726.0,-19557.0,190850.0,-21.75,30.0,X-LC,4.63,19.0,-0.25,1.42,12.13,X40N,ATH,FMCG
74,TITAGARH,1548.00,-2.05,-28.64,104.22,45.74,180117.0,-69353.0,172824.0,-4.90,26.0,H-SC,4.03,174.0,-0.39,1.29,12.40,XY24,NTT,ENGINEERING
72,TATAELXSI,9161.00,-1.90,-29.60,87.53,32.03,81241.0,-39020.0,92815.0,-23.76,28.0,H-SC,8.37,148.0,-0.48,0.69,3.96,OX40N,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.48,-6.63,122.67,107.91,169108.0,-9788.0,137856.0,-24.20,39.0,M-SC,12.17,216.0,-0.06,1.03,2.79,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.01,-12.98,14.94,0.02,22694.0,-22659.0,151900.0,131.58,51.0,M-SC,16.60,234.0,-1.00,1.13,32.13,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,1.25,-17.00,20.60,0.09,17231.0,-17133.0,83644.0,94.24,40.0,M-SC,2.36,253.0,-0.99,0.62,9.89,OX40N,NTT,DURABLES
66,SIS,528.00,0.27,-24.45,60.51,21.26,50757.0,-27150.0,83882.0,1978.76,47.0,H-SC,4.70,166.0,-0.53,0.62,13.33,OX40N,NTT,MISC
47,KANSAINER,340.00,0.83,-25.32,51.96,13.47,104634.0,-68292.0,201375.0,-69.28,30.0,H-SC,3.66,158.0,-0.65,1.50,2.39,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.47,6.07,62.41,72.27,112172.0,10286.0,179734.0,-8.8,66.0,M-LC,2.08,99.0,0.09,1.34,12.62,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.47,6.07,62.41,72.27,112172.0,10286.0,179734.0,-8.80,66.0,M-LC,2.08,99.0,0.09,1.34,12.62,XR,NTT,IT
38,INDIAMART,4810.62,-0.92,-3.87,115.05,106.72,136404.0,-4775.0,118561.0,-52.34,31.0,H-SC,1.58,139.0,-0.04,0.88,18.60,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.82,-2.17,38.63,35.63,79204.0,-4541.0,205033.0,-15.85,41.0,H-MC,3.15,119.0,-0.06,1.53,14.14,AR,ATH,PHARMA
25,FINCABLES,1641.55,1.48,-6.63,122.67,107.91,169108.0,-9788.0,137856.0,-24.20,39.0,M-SC,12.17,216.0,-0.06,1.03,2.79,OX40N,ATH,CABLES
37,IEX,219.00,-0.65,-4.81,56.13,48.63,106841.0,-9608.0,190346.0,-36.71,46.0,H-SC,14.08,136.0,-0.09,1.42,6.19,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-1.81,-14.76,42.64,21.58,50933.0,-20691.0,119448.0,-57.05,22.0,X-MC,6.09,56.0,-0.41,0.89,5.75,X40N,ATH,IT
9,BAJAJHFL,181.50,0.60,-21.40,88.87,48.45,140063.0,-42902.0,157604.0,-26.96,25.0,X-MC,11.68,64.0,-0.31,1.17,2.62,X40N,ATH,FINANCE
75,TMPV,600.00,-0.26,-18.53,74.52,42.18,167811.0,-51221.0,225189.0,-26.74,25.0,X-LC,5.61,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
33,HINDUNILVR,2922.00,-0.37,-10.61,27.16,13.67,62411.0,-27273.0,229790.0,-17.28,28.0,X-LC,2.78,14.0,-0.44,1.71,8.48,XY25,NTT,FMCG
53,PAGEIND,51605.08,-0.60,-9.69,39.59,26.07,58546.0,-15860.0,147880.0,-33.51,29.0,X-MC,11.86,55.0,-0.27,1.10,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,0.30,-21.89,96.52,53.50,227492.0,-66057.0,235694.0,-62.23,32.0,X-MC,1.34,58.0,-0.29,1.76,4.69,XY24,NTT,FMCG
15,CAMS,4762.0,-0.81,-80.52,540.57,24.79,265225.0,-202792.0,49064.0,-81.32,38.0,X-SC,1.52,86.0,-0.76,0.37,19.94,X40N,NTT,MISC
43,ITC,452.0,0.56,-1.46,12.08,10.44,23823.0,-2924.0,197214.0,-41.15,47.0,X-LC,1.71,5.0,-0.12,1.47,4.02,X40,NTT,FMCG
36,ICICIPRULI,790.0,3.11,6.60,22.86,30.97,43068.0,11661.0,188399.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.40,19.99,X40,ATH,INSURANCE
3,ACC,3906.0,-0.06,-25.03,118.95,64.15,212207.0,-59551.0,178400.0,-56.31,35.0,X-SC,1.98,82.0,-0.28,1.33,0.07,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.08,-0.60,-9.69,39.59,26.07,58546.0,-15860.0,147880.0,-33.51,29.0,X-MC,11.86,55.0,-0.27,1.10,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.26,-18.53,74.52,42.18,167811.0,-51221.0,225189.0,-26.74,25.0,X-LC,5.61,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
54,PGHH,17907.65,-0.05,-5.95,42.21,33.76,79726.0,-11940.0,188880.0,-33.72,34.0,X-MC,5.16,57.0,-0.15,1.41,0.04,X40,ATH,FMCG
3,ACC,3906.00,-0.06,-25.03,118.95,64.15,212207.0,-59551.0,178400.0,-56.31,35.0,X-SC,1.98,82.0,-0.28,1.33,0.07,XY24,BTT,CEMENT
31,HAVELLS,2069.16,-0.78,-9.81,46.88,32.48,138684.0,-32164.0,295827.0,-14.34,38.0,X-MC,5.28,67.0,-0.23,2.20,0.50,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.37,-30.22,104.70,42.84,47493.0,-19645.0,45361.0,-54.27,40.0,X-SC,36.83,83.0,-0.41,0.34,2.63,XY24,NTT,MISC
15,CAMS,4762.00,-0.81,-80.52,540.57,24.79,265225.0,-202792.0,49064.0,-81.32,38.0,X-SC,1.52,86.0,-0.76,0.37,19.94,X40N,NTT,MISC
59,RELAXO,1176.00,-1.59,-46.36,183.92,52.31,143219.0,-67290.0,77870.0,-42.89,48.0,X-SC,6.13,91.0,-0.47,0.58,4.23,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.18,-36.40,112.58,35.20,92133.0,-46832.0,81838.0,7.29,40.0,X-SC,15.40,92.0,-0.51,0.61,3.95,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.81,-14.76,42.64,21.58,50933.0,-20691.0,119448.0,-57.05,22.0,X-MC,6.09,56.0,-0.41,0.89,5.75,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.79,-13.51,37.92,19.28,113457.0,-46754.0,299202.0,-24.60,58.0,X-LC,5.50,1.0,-0.41,2.23,10.61,X40,ATH,IT
41,INFY,2275.00,-0.37,8.45,42.80,54.87,145233.0,26442.0,339330.0,-15.65,62.0,X-LC,3.30,2.0,0.18,2.53,17.43,X40,BTT,IT
75,TMPV,600.00,-0.26,-18.53,74.52,42.18,167811.0,-51221.0,225189.0,-26.74,25.0,X-LC,5.61,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
81,VBL,671.64,0.17,-4.59,42.19,35.67,127143.0,-14483.0,301359.0,-15.92,53.0,X-LC,4.73,4.0,-0.11,2.24,8.63,X40N,ATH,FMCG
43,ITC,452.00,0.56,-1.46,12.08,10.44,23823.0,-2924.0,197214.0,-41.15,47.0,X-LC,1.71,5.0,-0.12,1.47,4.02,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,2.79,-17.31,119.69,81.67,92851.0,-16234.0,77576.0,6972.22,42.0,L-SC,14.32,271.0,-0.17,0.58,50.23,XR,NTT,CERAMICS
50,MASFIN,398.61,0.39,-2.56,25.25,22.05,24107.0,-2505.0,95475.0,-15.99,55.0,H-SC,6.85,164.0,-0.10,0.71,38.78,XR,ATH,FINANCE
13,BSOFT,831.70,-1.29,-22.98,95.76,50.79,103744.0,-32316.0,108337.0,-1.04,66.0,H-SC,5.82,151.0,-0.31,0.81,26.14,XR,ATH,IT
39,INDIGOPNTS,1408.00,2.01,-12.98,14.94,0.02,22694.0,-22659.0,151900.0,131.58,51.0,M-SC,16.60,234.0,-1.00,1.13,32.13,OX40N,NTT,PAINTS
36,ICICIPRULI,790.00,3.11,6.60,22.86,30.97,43068.0,11661.0,188399.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.40,19.99,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-1.29,-22.98,95.76,50.79,103744.0,-32316.0,108337.0,-1.04,66.0,H-SC,5.82,151.0,-0.31,0.81,26.14,XR,ATH,IT
32,HCLTECH,1922.01,0.30,7.90,15.60,24.73,40720.0,19104.0,261028.0,12.76,63.0,X-LC,5.97,7.0,0.47,1.94,24.34,X40,ATH,IT
84,WIPRO,420.00,0.47,6.07,62.41,72.27,112172.0,10286.0,179734.0,-8.80,66.0,M-LC,2.08,99.0,0.09,1.34,12.62,XR,NTT,IT
56,RAJESHEXPO,518.00,0.61,-63.38,173.25,0.07,87034.0,-86941.0,50236.0,1740.91,54.0,L-SC,2.30,268.0,-1.00,0.37,24.94,OX40N,NTT,JEWELLERY
39,INDIGOPNTS,1408.00,2.01,-12.98,14.94,0.02,22694.0,-22659.0,151900.0,131.58,51.0,M-SC,16.60,234.0,-1.00,1.13,32.13,OX40N,NTT,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.71
1,25,45.71
2,50,76.63


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.06
MC    30.35
LC    24.60
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.29
X40      22.50
X40N     10.87
XR       10.10
XY25      9.36
AR        9.32
OX40N     7.80
SR        1.03
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.46
X-MC    23.58
X-LC    21.03
M-SC    12.15
X-SC     7.02
H-MC     5.00
L-SC     1.43
M-MC     1.41
M-LC     1.34
H-LC     1.18
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.37,-7.95,42.87
IT,12.77,-17.57,81.14
FINANCE,9.53,-21.09,72.26
ELECTRICAL,6.24,-11.82,52.78
PAINTS,6.12,-12.44,29.15
MISC,6.08,-59.17,124.38
INSURANCE,4.65,-1.30,37.00
PHARMA,4.11,-1.32,33.74
AUTO,3.40,-25.94,79.94


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3240374.0,21
AR,1319625.0,10
XR,1307359.0,13
X40,1018404.0,14
X40N,987709.0,9
OX40N,743624.0,10
XY25,384438.0,6
SR,282763.0,2
MH,84636.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3694336.0,25
M-SC,1426543.0,15
X-MC,1316204.0,16
X-SC,979230.0,8
X-LC,939102.0,11
H-MC,407492.0,3
L-SC,269644.0,3
M-LC,112172.0,1
H-LC,74765.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1292593.0      6
           AR         904001.0      5
M-SC       XY24       813987.0      6
H-SC       XR         784172.0      7
X-SC       X40N       504717.0      3
X-MC       X40        456539.0      7
           XY24       394919.0      3
X-LC       X40        381336.0      5
H-SC       OX40N      346171.0      4
M-SC       OX40N      310419.0      5
X-SC       XY24       293984.0      3
H-SC       SR         282763.0      2
X-MC       X40N       277123.0      4
X-LC       XY24       249865.0      2
H-MC       AR         212466.0      2
X-LC       X40N       205869.0      2
H-MC       XY24       195026.0      1
X-MC       XY25       187623.0      2
L-SC       XR         182610.0      2
X-SC       X40        180529.0      2
M-SC       XR         173744.0      2
           AR         128393.0      2
M-LC       XR         112172.0      1
X-LC       XY25       102032.0      2
L-SC       OX40N       87034.0      1
H-SC       MH          84636.0      1
H-LC       AR          74765.0      1
M-MC       XY25        55749.0      1
L-MC       XR          54661.0      1
L-LC       XY25        39034.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
